In [88]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import os



In [89]:
import torch
print(torch.cuda.is_available())

True


In [90]:
TRAIN = True

In [91]:
# df = pd.read_csv('test.csv', header=None, names=['haiku'], delimiter="@")
df = pd.read_parquet('dataset2.parquet')
df = df.rename(columns={"text":"haiku"})

In [92]:
df_sorted_gruen = df.sort_values(by='gruen_score', ascending=False)
df_sorted_gruen.head(10)

,source,haiku,text_phonemes,keywords,keyword_phonemes,gruen_score,text_punc
5280,bfbarry,What did you order? / I forgot what I wanted. ...,waht dihd yuw aor|der / ay fer|gaat waht ay wa...,you order,yuw aor|der,0.898392,None
1216,bfbarry,My lover is gone. / Won't you come back to my ...,may lah|ver axz gaon / wownt yuw kahm baek tax...,come back,kahm baek,0.896483,None
9866,twaiku,I just remembered. / I'm working tomorrow nigh...,ay jhahst rax|mehm|berd / ihm wer|kaxng tax|ma...,working tomorrow,wer|kihng tax|maa|row,0.894500,None
18866,twaiku,Your eyes deceive you. / An illusion fools you...,yaor ayz dax|siyv yuw / axn ax|luw|zhaxn fuwlz...,illusion fools,ax|luw|zhaxn fuwlz,0.893575,None
717,bfbarry,Worker bees can leave. / Even drones can fly a...,wer|ker biyz kaen liyv / iy|vaxn drownz kaen f...,worker bees,wer|ker biyz,0.893551,None
48902,haiku_data_2,Moonrise. / An owl swoops up. / Something.,muwn|rayz axn awl swuwps ahp sahm|thaxng,owl,awl,0.893530,Moonrise. An owl swoops up. Something.
35750,haiku_data_1,Moonrise. / An owl swoops up. / Something.,muwn|rayz axn awl swuwps ahp sahm|thaxng,owl,awl,0.893530,Moonrise. An owl swoops up. Something.
3741,bfbarry,Change begins right now. / Let music and dance...,cheynjh bax|gihnz rayt naw / leht myuw|zaxk ae...,dance commence,daens kax|mehns,0.893291,None
978,bfbarry,Winter is coming. / Cold winds are blowing ove...,wihn|ter axz kah|maxng / kowld wihndz aar blow...,cold winds,kowld wihndz,0.891717,None
2056,bfbarry,Your coldness burns me. / You stab me with ici...,yaor kowld|naxs bernz miy / yuw staeb miy wihd...,icicles,ay|sax|kaxlz,0.891561,None


In [93]:
df.head()

,source,haiku,text_phonemes,keywords,keyword_phonemes,gruen_score,text_punc
0,bfbarry,Delicate savage. / You'll never hold the cinde...,deh|lax|kaxt sae|vaxjh / yuwl neh|ver hhowld d...,cinder,sihn|der,0.639071,None
1,bfbarry,A splash and a cry. / Words pulled from the ri...,ax splaesh aend ax kray / werdz puhld frahm dh...,the riverside,dhax rih|ver|sayd,0.563353,None
2,bfbarry,"Steamy, mist rising. / Rocks receiving downwar...",stiy|miy mihst ray|zaxng / raaks rax|siy|vaxng...,mist rising,mihst ray|zaxng,0.538326,None
3,bfbarry,You were broken glass. / But I touched you eve...,yuw wer brow|kaxn glaes / baht ay tahcht yuw i...,broken glass,brow|kaxn glaes,0.703446,None
4,bfbarry,Eyes dance with firelight. / The Moon and I ar...,ayz daens wihdh faxr|layt / dhax muwn aend ay ...,eyes dance,ayz daens,0.830985,None


In [94]:
df.shape

(49024, 7)

In [95]:
df_test = df.sample(n = int(0.2 * len(df)))
df_train = df.loc[~df.index.isin(df_test.index)]

#Reset the indexes
# df_test = df_test.reset_index()
# df_train = df_train.reset_index()

In [96]:
df_train.shape

(39220, 7)

In [97]:
df_test.shape

(9804, 7)

In [98]:
# class Haiku(Dataset):
#     def __init__(self, control_code, truncate=False, gpt2_type="gpt2", max_length=1024):

#         self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
#         self.haiku = []

#         for row in df['haiku']:
#             self.haiku.append(torch.tensor(
#                 self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
#             ))
                
#         if truncate:
#             self.haiku = self.haiku[:20000]
            
#         self.haiku_count = len(self.haiku)
        
#     def __len__(self):
#         return self.haiku_count

#     def __getitem__(self, item):
#         return self.haiku[item]

class Haiku(Dataset):  # GPT suggested
    def __init__(self, control_code, truncate=False, gpt2_type="gpt2", max_length=1024):
        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.haiku = []

        for row in df['haiku']:
            # Ensure line breaks are preserved
            formatted_row = row[:-1].replace("/", "\\n")
            tokenized_haiku = self.tokenizer.encode(
                f"<|{control_code}|>{formatted_row}<|endoftext|>",
                truncation=True,
                max_length=max_length
            )
            self.haiku.append(torch.tensor(tokenized_haiku))
                
        if truncate:
            self.haiku = self.haiku[:20000]
            
        self.haiku_count = len(self.haiku)
        
    def __len__(self):
        return self.haiku_count

    def __getitem__(self, item):
        return self.haiku[item]

In [99]:
haiku = Haiku('haiku', truncate=True, gpt2_type="gpt2")

In [100]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [101]:
# Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

In [102]:
def train(
    dataset, model, tokenizer,
    batch_size=24, epochs=10, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir=".", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):

    acc_steps = 100
    device=torch.device("cuda")
    model = model.cuda()
    model.train()
    
    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):
        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [ ]:
#Train the model on the specific data we have
if TRAIN:
    model = train(haiku, model, tokenizer)

c:\Users\IBDA\.conda\envs\Tugas_akhir_stephen\lib\site-packages\transformers\optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training epoch 0
0


20000it [00:31, 635.29it/s]


Training epoch 1
tensor(3.5828, device='cuda:0', grad_fn=<NllLossBackward0>)


20000it [00:31, 635.09it/s]


...

20000it [00:31, 634.04it/s]


Training epoch 4
tensor(2.6696, device='cuda:0', grad_fn=<NllLossBackward0>)


20000it [00:30, 648.05it/s]


Training epoch 5
tensor(2.6845, device='cuda:0', grad_fn=<NllLossBackward0>)


2836it [00:04, 679.94it/s]

In [ ]:
#Save the model to a pkl or something so it can be reused later on
if TRAIN:
    torch.save(model, 'model.pt')

In [ ]:
model = torch.load('model.pt')

In [ ]:
model_prev = GPT2LMHeadModel.from_pretrained('gpt2')

In [ ]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=30, #maximum number of words
    top_p=0.8,
    temperature=1.,
):

    model.eval()

    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False

            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:
                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
                output_list = list(generated.squeeze().numpy())
                output_text = f"{tokenizer.decode(output_list)}<|endoftext| >" 
                generated_list.append(output_text)
                
    return generated_list

In [ ]:
x = generate(model.to("cpu"), tokenizer, "<|haiku|>A peashooter", entry_count=1)
print(x)

100%|██████████| 1/1 [00:02<00:00,  2.87s/it]

['<|haiku|>A peashooter? \\n To paint the ground to see. \\n Look at it fresh in my heart. \\n Peace without pain<|endoftext|>']


In [ ]:
print(generate(model_prev.to("cpu"), tokenizer, "<|haiku|>A peashooter", entry_count=1))

100%|██████████| 1/1 [00:03<00:00,  3.19s/it]

['Volcanic ashes rise from the ground, while countless other fires will erupt from the sky.\n\nDuring the first stage of a fire, people will see the heavenly bodies<|endoftext| >']


In [ ]:
# Function to generate multiple sentences. Test data should be a dataframe
def text_generation(test_data, model_type):
    generated_haiku = []
    for i in range(len(test_data[:10])):
        x = generate(model_type.to("cpu"), tokenizer, f"<|haiku|>{test_data["haiku"].iloc[i]}", entry_count=1)
        generated_haiku.append(x)
    return generated_haiku

In [ ]:
generated_haiku = text_generation(df_test, model)

100%|██████████| 1/1 [00:00<00:00,  1.19it/s]


In [ ]:
print(generated_haiku)

[['Just wanna say I. / Really like you and I hope. / U feel the same way.<|endoftext|>'], ['Night fading away. / New snow in the playground. / Seats. No shade<|endoftext|>'], ['Dawn light. / The bare shadow of a tree. / Flickers on the wall.\n\nHappy, and a little cry<|endoftext|>'], ["Crow's cawing. / We add a little bourbon. / To the pecan pie.<|endoftext|>"], ['My children and I. / Race to see the sunset. / Before it fades.<|endoftext|>'], ["Without. / A sound. / Killing frost? Um Who'll win the winter game.<|endoftext|>"], ["My son's birthday. / A red poinsettia leaf. / Flutters to the ground. / Everything worth talking about. / Seems to be trying to not listen<|endoftext|>"], ['Winter silence. / Another armload. / Of junk mail. / More boobs, more arms, more dick<|endoftext|>'], ['Twitter, can you get? / Us a new president soon. / Asking for a friend.<|endoftext|>'], ["One day they're going. / To call a holding on the. / Patriots one day. All we can do is go.<|endoftext|>"]]


## Generate Before Train

In [ ]:
generated_before = text_generation(df_test, model_prev)
generated_before

## Generate After Train

In [ ]:
generated_after = text_generation(df_test, model)
generated_after

---

In [ ]:
# # Loop to keep only generated text and add it as a new column in the dataframe
# my_generations = []

# for i in range(len(generated_haiku)):
#     a = df_test["haiku"][i].split()[-30:]  # Get the matching string we want (30 words)
#     b = " ".join(a)
#     c = " ".join(generated_haiku[i])  # Get all that comes after the matching string
#     my_generations.append(c.split(b)[-1])

# df_test["generated_haiku"] = my_generations

In [ ]:
# # Finish the sentences when there is a point, remove after that
# final = []

# for i in range(len(df_test)):
#     to_remove = df_test["generated_haiku"][i].split(".")[-1]
#     final.append(df_test["generated_haiku"][i].replace(to_remove, ""))

# df_test["generated_haiku"] = final
# df_test.head()

In [ ]:
# df_test['generated_haiku'][7]

In [ ]:
# df_test['haiku'][7]

# GRUEN for Evaluating Linguistic Quality of Generated Text

**Authors:** Wanzheng Zhu, Suma Bhat

**Published:** 2020

**Conference:** Proceedings of the 2020 Conference on Empirical Methods in Natural Language Processing: Findings

**Pages:** 94-108